#### Business Scenario
Manager would like to have a tool for accurate forecast of next-month revenue, to support AAVAIL while transitioning wolrdwide to an à-la-carte approach for customer pricing.


The ideal data for this would be historical revenue data for those countries who already adpoted the new pricing model, combined with independent variables which could be positively correlated to the revenue, such as service utilization, seasonality, amount of new platform content released and strength of marketing campaign.

In [10]:
import json
import pandas as pd
import os
import matplotlib.pyplot as plt 

In [26]:
data = []
for file in os.listdir("../cs-train/"):
    with open(os.path.join("../cs-train", file)) as f:
        data.append(pd.DataFrame.from_dict(json.load(f)))

In [28]:
for df in data:
    df.columns = ["Country", "Customer ID", "Invoice", "Price", "Stream ID", "Times Viewed", "Year", "Month", "Day"]

In [29]:
df = pd.concat(data)
df

,Country,Customer ID,Invoice,Price,Stream ID,Times Viewed,Year,Month,Day
0,United Kingdom,13085.0,489434,6.95,85048,12,2017,11,28
1,United Kingdom,NaN,489597,8.65,22130,1,2017,11,28
2,United Kingdom,NaN,489597,1.70,22132,6,2017,11,28
3,United Kingdom,NaN,489597,1.70,22133,4,2017,11,28
4,United Kingdom,NaN,489597,0.87,22134,1,2017,11,28
...,...,...,...,...,...,...,...,...,...
42013,United Kingdom,15628.0,562163,0.85,21544,12,2019,07,31
42014,United Kingdom,15628.0,562163,1.95,47591D,8,2019,07,31
42015,United Kingdom,15628.0,562163,0.83,23167,12,2019,07,31
42016,United Kingdom,15628.0,562163,4.95,22138,3,2019,07,31


In [25]:
len(df["Country"].unique())

43

In [30]:
df["date"] = pd.to_datetime(df[["Year", "Month", "Day"]])

In [31]:
df

,Country,Customer ID,Invoice,Price,Stream ID,Times Viewed,Year,Month,Day,date
0,United Kingdom,13085.0,489434,6.95,85048,12,2017,11,28,2017-11-28
1,United Kingdom,NaN,489597,8.65,22130,1,2017,11,28,2017-11-28
2,United Kingdom,NaN,489597,1.70,22132,6,2017,11,28,2017-11-28
3,United Kingdom,NaN,489597,1.70,22133,4,2017,11,28,2017-11-28
4,United Kingdom,NaN,489597,0.87,22134,1,2017,11,28,2017-11-28
...,...,...,...,...,...,...,...,...,...,...
42013,United Kingdom,15628.0,562163,0.85,21544,12,2019,07,31,2019-07-31
42014,United Kingdom,15628.0,562163,1.95,47591D,8,2019,07,31,2019-07-31
42015,United Kingdom,15628.0,562163,0.83,23167,12,2019,07,31,2019-07-31
42016,United Kingdom,15628.0,562163,4.95,22138,3,2019,07,31,2019-07-31


In [33]:
df["Invoice"].value_counts()

537434    1350
538071    1304
537638    1202
537237    1194
536876    1186
          ... 
528953       1
504625       1
553163       1
512169       1
520332       1
Name: Invoice, Length: 42646, dtype: int64

In [22]:
df.groupby(by="Country").agg({"Price": "sum"}).sort_values(by="Price", ascending=False)

,Price
Country,
United Kingdom,3.521514e+06
EIRE,1.070692e+05
Germany,4.927182e+04
France,4.056514e+04
Norway,3.849475e+04
Spain,1.604099e+04
Hong Kong,1.445257e+04
Portugal,1.352867e+04
Singapore,1.317592e+04


The available data contains monthly data on customer pricing, i.e. how much customers have been charged each month. This variable is directly correlated with the expected revenue, and it is thus here chosen as the target variable to be forecasted by our models.


The data spans a bit less than 2 years for 42 countries + Unspecified ones.
Given the prototypical nature of the solution we will restrict ourselves to predicting the next-month total pricing for the countries with the highest revenue.

In [34]:
df = df.sort_values(by="date").reset_index(drop=True)
df.to_csv("../combined.csv")